# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [101]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key
print(geoapify_key)

1025abf414844e2ba468f927316a64f6


## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [102]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv('WeatherPy_vacation-backup.csv')

# Display sample data
vacation_df.head(60)

,city_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,port alfred,ZA,75.18,broken clouds,-33.5906,26.8910,No hotel found
1,2,puerto ayora,EC,80.56,overcast clouds,-0.7393,-90.3518,Hostal La Mirada De Solitario George
2,4,mahebourg,MU,79.25,scattered clouds,-20.4081,57.7000,Grand Bel Air
3,5,yenagoa,NG,91.13,overcast clouds,4.9247,6.2642,Peretimi Hotels Ltd.
4,6,arrecife,ES,64.38,broken clouds,28.9630,-13.5477,Hostal San Gines
5,7,port lincoln,AU,63.48,overcast clouds,-34.7333,135.8667,Boston Hotel
6,8,la asuncion,VE,83.93,clear sky,11.0333,-63.8628,Hotel La Ciudad
7,9,pundaguitan,PH,80.58,overcast clouds,6.3711,126.1689,No hotel found
8,10,chincholi,IN,70.45,clear sky,17.4667,77.4333,Limra Hotel
9,13,cabedelo,BR,84.87,broken clouds,-6.9811,-34.8339,No hotel found


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [106]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM"
)


# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [107]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df['City'] == 'bredasdorp']
vacation_end = vacation_df.loc[vacation_df['City'] == 'bredasdorp']
vacation_stop1 = vacation_df.loc[vacation_df['City'] == 'cape town']
vacation_stop2 = vacation_df.loc[vacation_df['City'] == 'richards bay']
vacation_stop3 = vacation_df.loc[vacation_df['City'] == 'hermanus']
vacation_start
vacation_end
vacation_stop1
vacation_stop2
vacation_stop3

,city_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
50,90,hermanus,ZA,80.8,few clouds,-34.4187,19.2345,Aloe guest house


## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [108]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, 
                          vacation_stop2, vacation_stop3]
                         ,)

# Display sample data
itinerary_df.head()

,city_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,15,bredasdorp,ZA,72.10,overcast clouds,-34.5322,20.0403,Victoria Hotel
49,89,cape town,ZA,78.91,clear sky,-33.9258,18.4232,Townhouse Hotel
69,129,richards bay,ZA,79.36,scattered clouds,-28.7830,32.0377,Protea Hotel
50,90,hermanus,ZA,80.80,few clouds,-34.4187,19.2345,Aloe guest house


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [109]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df =itinerary_df[["Lat", "Lng"]]

# Display sample data
waypoints_df 

,Lat,Lng
11,-34.5322,20.0403
49,-33.9258,18.4232
69,-28.7830,32.0377
50,-34.4187,19.2345


## Use GeoViews to create map that shows the four cities in the itinerary

In [110]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM"
)






    


In [111]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [112]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [113]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '1025abf414844e2ba468f927316a64f6',
 'waypoints': '-34.5322,20.0403|-33.9258,18.4232|-28.783,32.0377|-34.4187,19.2345'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [94]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"



# Make request and retrieve the JSON data by using the params dictionaty

route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format

route_response = route_response.json()
route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [20.0403, -34.5322], 'original_index': 0},
     {'location': [18.4232, -33.9258], 'original_index': 1},
     {'location': [32.0377, -28.783], 'original_index': 2},
     {'location': [19.2345, -34.4187], 'original_index': 3}],
    'units': 'metric',
    'distance': 3808634,
    'distance_units': 'meters',
    'time': 125808.651,
    'legs': [{'distance': 185797,
      'time': 6571.194,
      'steps': [{'from_index': 0,
        'to_index': 922,
        'distance': 72620,
        'time': 2403.621,
        'instruction': {'text': 'Drive northwest on Lang Street/R316/Long Street. Continue on R316.'}},
       {'from_index': 922,
        'to_index': 3077,
        'distance': 111791,
        'time': 4071.307,
        'instruction': {'text': 'Turn left toward Cape Town.'}},
       {'from_index': 3077,
        'to_index': 3095,
        'distance': 766,
        'time': 45.375,
        'instruction'

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [95]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]["coordinates"]

legs


[[[20.040159, -34.532389],
  [20.039917, -34.532266],
  [20.039738, -34.532158],
  [20.039222, -34.531858],
  [20.039153, -34.53182],
  [20.038922, -34.531674],
  [20.038526, -34.531444],
  [20.038047, -34.531198],
  [20.037535, -34.530895],
  [20.037371, -34.53081],
  [20.036972, -34.530594],
  [20.036746, -34.530457],
  [20.036629, -34.530393],
  [20.036596, -34.530372],
  [20.036536, -34.530317],
  [20.036409, -34.530174],
  [20.036172, -34.529888],
  [20.035954, -34.529622],
  [20.035663, -34.52926],
  [20.035381, -34.528945],
  [20.035122, -34.528647],
  [20.034792, -34.528258],
  [20.034403, -34.527865],
  [20.034224, -34.527712],
  [20.033934, -34.527491],
  [20.033626, -34.527272],
  [20.033438, -34.527136],
  [20.033312, -34.527055],
  [20.032261, -34.52643],
  [20.031965, -34.526256],
  [20.031882, -34.526201],
  [20.031344, -34.525879],
  [20.030834, -34.525537],
  [20.030442, -34.525215],
  [20.030204, -34.524957],
  [20.029853, -34.524513],
  [20.029298, -34.523764],
  [20

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [114]:
# Create and empty list to store the longitude of each step
Lng = []

# Create and empty list to store the latitude of step
Lat = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for leg in legs:
    for coordinate in leg:
        Lng.append(coordinate[0])
        Lat.append(coordinate[1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [115]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df["Longitude"]= Lng
route_df["Latitude"]= Lat
                    

# Display sample data
route_df.head()

,Longitude,Latitude
0,20.040159,-34.532389
1,20.039917,-34.532266
2,20.039738,-34.532158
3,20.039222,-34.531858
4,20.039153,-34.531820


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [116]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(
    route_df,
).opts(
    height= 500,
    width = 700,
    line_width = 3,
    color = "RED"
)



In [117]:
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

In [118]:
route_path

:Path   [Longitude,Latitude]

In [119]:
# Display a composed plot by using the route_map and route_path objects
map_plot*route_path

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]
   .Path.I   :Path   [Longitude,Latitude]